<a href="https://colab.research.google.com/github/Levelleor/googlecollab/blob/main/ComfyUIonColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#`mountUmount(`<font size="3px" color="#01c968">`Gdrive`</font>`)`



In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount Gdrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive!
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

#`Setup And Update ComfyUI`



In [1]:
from pathlib import Path

OPTIONS = {}

DRIVE_PATH = ""  # @param {type:"string"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
WORKSPACE = '/content/ComfyUI'
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if DRIVE_PATH:

    WORKSPACE = DRIVE_PATH+"/ComfyUI"
    %cd {DRIVE_PATH}

![ ! -d WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

%cd {WORKSPACE}

-= Initial setup ComfyUI =-
Cloning into 'ComfyUI'...
remote: Enumerating objects: 29173, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 29173 (delta 47), reused 35 (delta 32), pack-reused 29101 (from 3)
Receiving objects: 100% (29173/29173), 75.51 MiB | 12.58 MiB/s, done.
Resolving deltas: 100% (19800/19800), done.
/content/ComfyUI
-= Updating ComfyUI =-
Already up to date.
-= Install dependencies =-
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121, https://download.pytorch.org/whl/cu118, https://download.pytorch.org/whl/cu117
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 114.1 MB/s 

# `Models Download`

## Download MODELS

In [ ]:
# --- 0. Cleanup Workspace ---
# Removes the default Colab sample_data folder immediately
!rm -rf /content/sample_data

%cd /content/ComfyUI/

# --- 1. System & Python Prerequisites ---
print("Installing System & Python Prerequisites...")

# Install Aria2 for fast downloads
!apt-get -y install -qq aria2

# Install Python dependencies
!pip install segment-anything ultralytics scikit-image piexif timm transformers GitPython opencv-python-headless

from google.colab import userdata

# Try to get the token
try:
  CIVITAI_API_TOKEN = userdata.get('CIVITAI_API_TOKEN')
except:
  CIVITAI_API_TOKEN = None

print("Loaded API key:", "✅" if CIVITAI_API_TOKEN else "❌ Not found")

# --- 2. Function Definitions ---

def install_custom_node(url):
  %cd /content/ComfyUI/custom_nodes
  !git clone {url}

def downloadModel(url, filename = None):
  # Check if it's a huggingface 'blob' link and convert to 'resolve' for download
  if 'huggingface.co' in url and '/blob/' in url:
    url = url.replace('/blob/', '/resolve/')

  if 'huggingface.co' in url:
    if filename is None:
      filename = url.split('/')[-1]
      filename = filename.removesuffix('?download=true')
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -o {filename}
  else:
    # civitai
    if filename:
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}?token={CIVITAI_API_TOKEN} -o {filename}
    else:
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}?token={CIVITAI_API_TOKEN}

# --- 3. Install Custom Nodes ---
print("Installing Custom Nodes...")

install_custom_node('https://github.com/ltdrdata/ComfyUI-Manager.git')
install_custom_node('https://github.com/ltdrdata/ComfyUI-Impact-Pack')
install_custom_node('https://github.com/ltdrdata/ComfyUI-Impact-Subpack')
install_custom_node('https://github.com/storyicon/comfyui_segment_anything')
install_custom_node('https://github.com/Kosinkadink/ComfyUI-Advanced-ControlNet')
install_custom_node('https://github.com/Fannovel16/comfyui_controlnet_aux')
install_custom_node('https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved')
install_custom_node('https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite')
install_custom_node('https://github.com/kijai/ComfyUI-KJNodes')
install_custom_node('https://github.com/JPS-GER/ComfyUI_JPS-Nodes')
install_custom_node('https://github.com/rgthree/rgthree-comfy')
install_custom_node('https://github.com/cubiq/ComfyUI_IPAdapter_plus')
install_custom_node('https://github.com/ssitu/ComfyUI_UltimateSDUpscale')
install_custom_node('https://github.com/kijai/ComfyUI-Florence2')
install_custom_node('https://github.com/kijai/ComfyUI-RMBG')
install_custom_node('https://github.com/sipherxyz/comfyui-art-venture')
install_custom_node('https://github.com/Kik00/ComfyUI-KikoTools')


# --- 4. Download Checkpoints ---
print("Downloading Checkpoints...")
%cd /content/ComfyUI/models/checkpoints

# CyberRealistic Pony
downloadModel('https://civitai.com/api/download/models/2071650')

# Additional requested checkpoints
downloadModel('https://civitai.com/api/download/models/2155386')
downloadModel('https://civitai.com/api/download/models/1759168')
downloadModel('https://civitai.com/api/download/models/1966530')
downloadModel('https://civitai.com/api/download/models/2442439')
downloadModel('https://civitai.com/api/download/models/2514955')


# --- 5. Download LoRAs ---
print("Downloading LoRAs...")
%cd /content/ComfyUI/models/loras

# dnd2 speed lora
downloadModel('https://civitai.com/api/download/models/1820705')


# --- 6. Download VAE ---
print("Downloading VAE...")
%cd /content/ComfyUI/models/vae
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors" -o "vae-ft-mse-840000-ema-pruned.safetensors"


# --- 7. Download ControlNet (Custom Structure) ---
print("Downloading Specialized ControlNet...")
# Creating the specific directory structure requested in the image: SDXL/controlnet-union-sdxl-1.0/
!mkdir -p /content/ComfyUI/models/controlnet/SDXL/controlnet-union-sdxl-1.0/
%cd /content/ComfyUI/models/controlnet/SDXL/controlnet-union-sdxl-1.0/

# Downloading Xinsir Promax and renaming
downloadModel('https://huggingface.co/xinsir/controlnet-union-sdxl-1.0/resolve/main/diffusion_pytorch_model_promax.safetensors', 'SDXL_all_in_one_model_promax.safetensors')


# --- 8. Download Upscaler ---
print("Downloading Upscaler...")
%cd /content/ComfyUI/models/upscale_models
downloadModel('https://huggingface.co/shubhdotai/upscaler/resolve/9b9bfba2a119bee0d175427cadb05283a3ed2fb1/4xNMKDSuperscale_4xNMKDSuperscale.pt')


%cd /content/ComfyUI/
print("Installation and Downloads Complete.")

[Errno 2] No such file or directory: '/content/ComfyUI/'
/content
Installing System & Python Prerequisites...
Loaded API key: ✅
Installing Custom Nodes...
[Errno 2] No such file or directory: '/content/ComfyUI/custom_nodes'
/content
fatal: destination path 'ComfyUI-Manager' already exists and is not an empty directory.
[Errno 2] No such file or directory: '/content/ComfyUI/custom_nodes'
/content
fatal: destination path 'ComfyUI-Impact-Pack' already exists and is not an empty directory.
[Errno 2] No such file or directory: '/content/ComfyUI/custom_nodes'
/content
fatal: destination path 'ComfyUI-Impact-Subpack' already exists and is not an empty directory.
[Errno 2] No such file or directory: '/content/ComfyUI/custom_nodes'
/content
fatal: destination path 'comfyui_segment_anything' already exists and is not an empty directory.
[Errno 2] No such file or directory: '/content/ComfyUI/custom_nodes'
/content
fatal: destination path 'ComfyUI-Advanced-ControlNet' already exists and is not an 

## OTHERS

In [4]:
# SD1.5
# !wget -c https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -P ./models/checkpoints/

# Some SD1.5 anime style
# !wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_hard.safetensors -P ./models/checkpoints/

# VAE
!wget -c https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -P ./models/vae/

--2025-12-30 02:27:16--  https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt
Resolving huggingface.co (huggingface.co)... 18.164.174.118, 18.164.174.23, 18.164.174.17, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.118|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /stable-diffusion-v1-5/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt [following]
--2025-12-30 02:27:17--  https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt
Reusing existing connection to huggingface.co:443.
HTTP request sent, awaiting response... 302 Found
Location: https://cas-bridge.xethub.hf.co/xet-bridge-us/66d19580e2632490a6bc5829/773d9e5ddbf6a84612066b3ea00a0bcfe4ba870a9befc88816f71b5927e76e38?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Content-Sha256=UNSIGNED-PAYLOAD&X-Amz-Credential=cas%2F20251230%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20251230T021742Z&

## LIST MODELS

In [5]:
!ls -al ./models/checkpoints/

total 16381104
drwxr-xr-x  2 root root       4096 Dec 30 02:27 .
drwxr-xr-x 23 root root       4096 Dec 30 02:13 ..
-rw-r--r--  1 root root 5570803627 Dec 30 02:28 AbyssOrangeMix2_hard.safetensors
-rw-r--r--  1 root root 6938040682 Dec 30 02:27 cyberrealisticPony_v127Alt.safetensors
-rw-r--r--  1 root root          0 Dec 30 02:13 put_checkpoints_here
-rw-r--r--  1 root root 4265380512 Dec 30 02:27 v1-5-pruned-emaonly.ckpt


# `START ComfyUI  & Expose Server (MANUAL)`

## Download Prerequisits

In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

In [ ]:
!python main.py --dont-print-server

# `START ComfyUI & Expose Server`

## Download Prerequisits

In [6]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

--2025-12-30 02:28:09--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64.deb [following]
--2025-12-30 02:28:09--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/8a32f7c6-649c-4f0d-806d-e14c19d0786d?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-30T03%3A19%3A53Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64.deb&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4d

## CF Tunnel

In [10]:
import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server --lowvram

Checkpoint files will always be loaded safely.
Total VRAM 15095 MB, total RAM 12976 MB
pytorch version: 2.9.1+cu128
xformers version: 0.0.33.post2
Set vram state to: LOW_VRAM
Device: cuda:0 Tesla T4 : cudaMallocAsync
Using async weight offloading with 2 streams
Enabled pinned memory 12326.0
working around nvidia conv3d memory bug.
Using xformers attention
Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
ComfyUI version: 0.6.0
ComfyUI frontend version: 1.35.9
[Prompt Server] web root: /usr/local/lib/python3.12/dist-packages/comfyui_frontend_package/static
Total VRAM 15095 MB, total RAM 12976 MB
pytorch version: 2.9.1+cu128
xformers version: 0.0.33.post2
Set vram state to: LOW_VRAM
Device: cuda:0 Tesla T4 : cudaMallocAsync
Using async weight offloading with 2 streams
Enabled pinned memory 12326.0

Import times for custom nodes:
   0.0 seconds: /content/ComfyUI/custom_nodes/websocket_image_save.py

Context impl SQLiteImpl.
Will assume non-transactional DDL.
No target rev

## localtunnel

In [8]:
# localtunnel
!npm install -g localtunnel

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()
%cd /content/ComfyUI
!python main.py --dont-print-server

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙
added 22 packages in 2s
⠙
⠙3 packages are looking for funding
⠙  run `npm fund` for details
⠙npm notice
npm notice New major version of npm available! 10.8.2 -> 11.7.0
npm notice Changelog: https://github.com/npm/cli/releases/tag/v11.7.0
npm notice To update run: npm install -g npm@11.7.0
npm notice
⠙/content/ComfyUI
Checkpoint files will always be loaded safely.
Total VRAM 15095 MB, total RAM 12976 MB
pytorch version: 2.9.1+cu128
xformers version: 0.0.33.post2
Set vram state to: NORMAL_VRAM
Device: cuda:0 Tesla T4 : cudaMallocAsync
Using async weight offloading with 2 streams
Enabled pinned memory 12326.0
working around nvidia conv3d memory bug.
Using xformers attention
Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
ComfyUI version: 0.6.0
ComfyUI frontend version: 1.35.9
[Prompt Server] web root: /usr/local/lib/python3.12/dist-packages/comfyui_frontend_package/static
Total VRAM 15095 MB, total RAM 12976 MB
pytorch version: 2.9.1+cu128
xformers